In [1]:
import json
import pandas as pd
import folium

In [2]:
csv_in = 'crime2006_2016-utf8.csv'
html_out = 'crime2006_2016-utf8.html'
geo = 'japan.json'
csv_pref = 'pref_roman-utf8.csv'

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [7]:
df_pop = pd.read_csv(csv_in,delimiter=',',skiprows=0,header=0)
df_pref = pd.read_csv(csv_pref,delimiter=',',skiprows=0,header=0)
df_merged = pd.merge(df_pop,df_pref,on='都道府県',how='left')
display(df_merged.head())

,年,都道府県,検挙人数,都道府県(名のみ),都道府県読み,Roman
0,2016,北海道,8460,北海道,ほっかいどう,Hokkaido
1,2016,青森県,1621,青森,あおもり,Aomori
2,2016,岩手県,1282,岩手,いわて,Iwate
3,2016,宮城県,3209,宮城,みやぎ,Miyagi
4,2016,秋田県,1364,秋田,あきた,Akita


In [9]:
print(df_merged.loc[df_merged['検挙人数'].idxmin()])
print(df_merged.loc[df_merged['検挙人数'].idxmax()])

年               2016
都道府県             島根県
検挙人数             858
都道府県(名のみ)         島根
都道府県読み           しまね
Roman        Shimane
Name: 31, dtype: object
年             2006
都道府県           東京都
検挙人数         58775
都道府県(名のみ)       東京
都道府県読み       とうきょう
Roman        Tokyo
Name: 477, dtype: object


In [12]:
csv_in2 = 'h3010pref_pos_utf8.csv'
df_pos = pd.read_csv(csv_in2, delimiter=',', skiprows=0, header=0)
df_merged2 = pd.merge(df_merged, df_pos, left_on='都道府県', right_on='name', how='left')
print(df_merged2.shape)
display(df_merged2.head())

(517, 9)


,年,都道府県,検挙人数,都道府県(名のみ),都道府県読み,Roman,name,lat,lon
0,2016,北海道,8460,北海道,ほっかいどう,Hokkaido,北海道,43.064278,141.346785
1,2016,青森県,1621,青森,あおもり,Aomori,青森県,40.824338,140.740087
2,2016,岩手県,1282,岩手,いわて,Iwate,岩手県,39.703647,141.152592
3,2016,宮城県,3209,宮城,みやぎ,Miyagi,宮城県,38.268803,140.871846
4,2016,秋田県,1364,秋田,あきた,Akita,秋田県,39.718058,140.103250


In [13]:
df_heat = df_merged2[['lat', 'lon', '検挙人数']]
print(df_heat.shape)
display(df_heat.head())

(517, 3)


,lat,lon,検挙人数
0,43.064278,141.346785,8460
1,40.824338,140.740087,1621
2,39.703647,141.152592,1282
3,38.268803,140.871846,3209
4,39.718058,140.103250,1364


In [14]:
ar_heat = []
for i,r in df_heat.iterrows():
    ar_heat.append(r)
print(ar_heat[:3])

[lat       43.064278
lon      141.346785
検挙人数    8460.000000
Name: 0, dtype: float64, lat       40.824338
lon      140.740087
検挙人数    1621.000000
Name: 1, dtype: float64, lat       39.703647
lon      141.152592
検挙人数    1282.000000
Name: 2, dtype: float64]


In [29]:
from folium.plugins import HeatMap
import branca.colormap as cm

m = folium.Map(location=[38, 140], zoom_start=4)
m.add_child(HeatMap(ar_heat, radius=9,
                    gradient={0:'blue', 0.5:'lime', 1:'red'}))

min = df_heat['検挙人数'].min()
max = df_heat['検挙人数'].max()
colors = ['blue', 'lime', 'red']
colormap = cm.LinearColormap(colors, vmin=min, vmax=max).to_step(10000)
colormap.caption = '検挙人数'
m.add_child(colormap)

display(m)
m.save(outfile='dm-03-assign2.html')